In [ ]:
!wc -l /home/wangxinglu/prj/few-shot/data/imglst/img10k.test.smpl 
!wc -l /home/wangxinglu/prj/few-shot/data/imglst/img10k.test.txt 

from utils import *
from metadata import *
from model_utils import *
from gen_imglst import *

import utils
%load_ext autoreload
# %reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

def run():
    pred,lb = get_prediction()
    print per_cls_acc(pred,lb)

def get_prediction(path='/mnt/nfs1703/test/prj/few-shot/models/res101.img10k.longtail.flatten3',name=None):
    if name is None:
        name = path.split('/')[-1]+'.h5'
    
    if not osp.exists(path+'/snapshots/iter.best.parrots'):
        utils.ln(path= path+'/snapshots/iter.latest.parrots',to_path=path+'/snapshots/iter.best.parrots')
    f=load_model(path)
    
    utils.rm(name)
    ff=h5py.File(name)
    ind=0
    while ind<568165:
#         try:
        f.feed()
#         append_file(str(ind),'val.ok')
#         except Exception as  inst :
#             print inst, ind
#             break

        f.forward()
#         lbs.append(f.data('label').value())
        #     datas.append(f.data('data').value())
#         fcs.append(f.data('luzai.cls').value())
        lbs=f.data('label').value()
        ff['lbs/'+str(ind)]=lbs
        ff['fcs/'+str(ind)]=f.data('prob').value()
        ind+=lbs[-1].shape[-1]

#     lb=np.concatenate(lbs,axis=-1)
#     fc=np.concatenate(fcs,axis=-1)

#     pred=np.argmax(fc,axis=0)
#     lb=lb.reshape((lb.shape[-1],))
    
    ff.close()
    return f 
    
#     return pred,lb

def per_cls_acc(pred,lb):
    ### per cls acc
    cnts,bounds,_=plt.hist(lb,bins=10000,range=(-0.5,1000-0.5))
    assert len(cnts[cnts==0])==0, 'val must have some samples'
    print(min(cnts))

    from sklearn.metrics import confusion_matrix
    conf = confusion_matrix(lb, pred,np.arange(0,10000)).astype(float)

    plt.figure()
    plt.plot(np.diag(conf)/cnts)

    plt.figure()
    cnts,bounds,_=plt.hist(lb,bins=1000,range=(-0.5,1000-0.5))
    markerline, stemlines, baseline =plt.stem(cnts*np.diag(conf)/cnts)
    _=plt.setp(baseline, 'color', 'b')
    _=plt.setp(stemlines,'color','b')
    # plt.plot(cnts*np.diag(conf)/cnts)

    return  np.mean(lb==pred) , np.nanmean(np.diag(conf)/cnts)
path='/home/wangxinglu/prj/few-shot/models/'

# f=get_prediction()
import multiprocessing as mp 

for d in[
#     'res101.img10k.longtail.flatten.undercomp',
#     'res101.img10k.longtail.flatten3',
#     'res101.img10k.longtail.flatten2',
    'res101.img10k.no1k.flatten.2048'
]:
    p=path+d
    n=d+'.h5'
    print p,n
    f=get_prediction(p,n)
    print 'ok'
    f.release()

In [ ]:
from utils import *
from metadata import *
from model_utils import *
from gen_imglst import *

import utils
%load_ext autoreload
# %reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

def run():
    pred,lb = get_prediction()
    print per_cls_acc(pred,lb)

def get_prediction(path='/mnt/nfs1703/test/prj/few-shot/models/res101.img10k.longtail.flatten3'):
    f=load_model(path)
    utils.rm('cache.h5')
    ff=h5py.File('cache.h5')
    ind=0
    while ind<129678:
#         try:
        f.feed()
#         append_file(str(ind),'val.ok')
#         except Exception as  inst :
#             print inst, ind
#             break

        f.forward()
#         lbs.append(f.data('label').value())
        #     datas.append(f.data('data').value())
#         fcs.append(f.data('luzai.cls').value())
        lbs=f.data('label').value()
        ff['lbs/'+str(ind)]=lbs
        ff['fcs/'+str(ind)]=f.data('data').value()
        ind+=lbs[-1].shape[-1]
    
    ff.close()
    f.close()
#     return pred,lb


def per_cls_acc(pred,lb):
    ### per cls acc
    cnts,bounds,_=plt.hist(lb,bins=10000,range=(-0.5,10000-0.5))
#     assert len(cnts[cnts==0])==0, 'val must have some samples'
    print  len(cnts[cnts==0])
    print(min(cnts))
#     cnts[cnts==0]+=1

    from sklearn.metrics import confusion_matrix
    conf = confusion_matrix(lb, pred,np.arange(0,10000)).astype(float)

    plt.figure()
    plt.plot(np.diag(conf)/cnts)

    plt.figure()
    cnts,bounds,_=plt.hist(lb,bins=10000,range=(-0.5,10000-0.5))
#     markerline, stemlines, baseline =plt.stem(cnts*np.diag(conf)/cnts)
#     _=plt.setp(baseline, 'color', 'b')
#     _=plt.setp(stemlines,'color','b')
    plt.plot(cnts*np.diag(conf)/cnts)
    print np.mean(lb==pred) , np.nanmean(np.diag(conf)/cnts)
    return  np.mean(lb==pred) , np.nanmean(np.diag(conf)/cnts)

def analyze(aname):
    aname+='.h5'
    if not osp.exists(aname): return  
    f=h5py.File(aname,'r')
    orders = f['fcs'].keys()

    lbs=[f['lbs/'+l].value for l in orders]
    fcs=[f['fcs/'+l].value for l in orders]

    lb=np.concatenate(lbs,axis=-1)
    fc=np.concatenate(fcs,axis=-1)

    pred=np.argmax(fc,axis=0)
    pred=pred.reshape((pred.shape[-1],))
    lb=lb.reshape((lb.shape[-1],))
    per_cls_acc(pred,lb)

for aname in [ 
    'res101.img10k.longtail.flatten.undercomp',
    'res101.img10k.longtail.flatten3',
    'res101.img10k.longtail.flatten2',
    'res101.img10k.no1k.flatten.2048'
] :
    print analyze(aname)

In [ ]:
get_prediction('/home/wangxinglu/prj/few-shot/models/res101.img10k.longtail.flatten.2048.chk')

testf='/home/wangxinglu/prj/few-shot/data/imglst/img10k.test.txt'
t=read_list(testf)

392580-1+109

len(t)
t[392580-1:392580+109][:5]

bak=os.getcwd()

sl=[]
ind=0
for imgf,lb in t[392580-1:392580+109]:
#     print ind
    img=cv2.imread(imgf,cv2.CV_LOAD_IMAGE_COLOR)
    cv2.imwrite(imgf,img,[int(cv2.IMWRITE_JPEG_QUALITY), 100])
    sl.append(img.shape)
    ind+=1
#     if ind==4:
#         break

